In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import Callback
import tensorflow_datasets as tfds
(dataset, dataset_info) = tfds.load('cats_vs_dogs', split='train[:80%]', with_info=True, as_supervised=True)
num_classes = dataset_info.features['label'].num_classes
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.2, upper=1.8)
    image = tf.image.per_image_standardization(image)
    label = tf.one_hot(label, num_classes)
    return image, label
dataset = dataset.map(preprocess_image)
num_samples = dataset_info.splits['train[:90%]'].num_examples
num_train_samples = int(0.8 * num_samples)
train_dataset = dataset.take(num_train_samples)
val_dataset = dataset.skip(num_train_samples)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
class TrainingAccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Training Accuracy: {logs['accuracy']:.4f}")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
accuracy_callback = TrainingAccuracyCallback()
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[accuracy_callback])
final_training_accuracy = history.history['accuracy'][-1]
print("Training accuracy:", final_training_accuracy)


Epoch 1/10
524/524 [==============================] - 131s 235ms/step - loss: 0.7113 - accuracy: 0.6372 - val_loss: 0.5710 - val_accuracy: 0.7058
Epoch 2/10
524/524 [==============================] - 116s 222ms/step - loss: 0.5507 - accuracy: 0.7234 - val_loss: 0.5170 - val_accuracy: 0.7543
Epoch 3/10
524/524 [==============================] - 113s 216ms/step - loss: 0.4813 - accuracy: 0.7688 - val_loss: 0.4719 - val_accuracy: 0.7717
Epoch 4/10
524/524 [==============================] - 116s 221ms/step - loss: 0.4173 - accuracy: 0.8122 - val_loss: 0.4506 - val_accuracy: 0.7872
Epoch 5/10
524/524 [==============================] - 111s 212ms/step - loss: 0.3608 - accuracy: 0.8423 - val_loss: 0.4446 - val_accuracy: 0.8068
Epoch 6/10
524/524 [==============================] - 112s 214ms/step - loss: 0.3059 - accuracy: 0.8731 - val_loss: 0.4517 - val_accuracy: 0.8130
Epoch 7/10
524/524 [==============================] - 111s 213ms/step - loss: 0.2447 - accuracy: 0.9032 - val_loss: 0.4676 -